## 导包

In [2]:
import sys
sys.path.append('../')
from utils import *
import json
import pymysql
import torch
read_con = pymysql.connect(host="192.168.126.143",port = 3337,user='root', password='pjzhang', database='ccks_2019',charset='utf8')
cur = read_con.cursor()

### 实体识别，融合暴力

In [3]:
questions=[]
with open("../data/test.json",'r')as f:
    all_test_data = json.load(f)
    questions = all_test_data[1]
    gold_mentions = all_test_data[6]
    
fn = "model_decode"
fn2 = 'data/questions_ws.txt'
fn_out = "data/test_er_out.json"
i = -1
with open(fn,'r',encoding='utf-8') as f:
    out=[]
    context=f.read().strip()
    lines=context.split('\n\n')
    B = True
    # 粗粒度
    if not B:
        for line in lines:
            i+=1
            # 一句话
            result=line.split('\n')
            mentions=[]
            m=''
            q=questions[i]

            assert len(result) == len(q)
            # 识别mention
            for r,word in zip(result,q):

                kkk=r.split('\t')
                try:
                    char_q,char_r=word,kkk[1]
                except:
                    pass
                if char_r!='O' and char_r!='O-':
                    m=m+char_q
                else:
                    if m:
                        mentions.append(m)
                    m=''
            out.append(mentions)

    #细粒度
    else:
        for line in lines:
            i+=1
            # 一句话
            result=line.split('\n')
            mentions=[]
            m=''
            # if len(result)==1 and not result[0]:
            #     continue
            q=questions[i]
            #print(result,q)
            assert len(result) == len(q)
            for r,word in zip(result,q):
                kkk=r.split('\t')
                try:
                    char_q,char_r=word,kkk[1]
                except:
                    pass
                    #print(line,result)
                if char_r=='B-':
                    if m:
                        mentions.append(m)
                    m=char_q
                elif char_r=='I-':
                    m=m+char_q
                else:
                    if m:
                        mentions.append(m)
                    m=''
            out.append(mentions)

# ans1识别出的实体
ans1=out
ans2=[] # 暴力搜索mention
ans3=[] # 最大匹配分词

# fn2 系统3的分词结果
with open(fn2,'r',encoding='utf-8') as f2:
    line_num=-1
    for line in f2:
        line = line.strip()
        line_num+=1
        question_index=int(line_num/3)
        # 只用了两种分词结果
        if line_num%3==0:
            #print(line,questions)
            assert line.strip()==questions[question_index]
            
        elif line_num%3==2:
            ans2.append(line.strip().split('\t'))
        elif line_num%3==1:
            ans3.append(line.strip().split('\t'))

ans4=[] # 最后结果
# topk=2
for a1,a2,a3,q in zip(ans1,ans2,ans3,questions):
    print(a1,a3,q)
    if not a1:
        ans4.append(a3)
    elif not a2:
        ans4.append(a1)
    else:
        a4=[]
        for i in a1:
            if i[0]=='"' and i[-1]=='"':
                i = i[1:-1]
            if i in a2:
                if len(i)>1:
                    if i[0]=='"' and i[-1]=='"':
                        i = i[1:-1]
                    a4.append(i)
            elif len(i)>1 :
                if i in a2:
                    if i[0]=='"' and i[-1]=='"':
                        i = i[1:-1]
                    a4.append(i)
        for i in a1:
            
            if i=="键盘繁琐":
                print(a1)
            start_i=q.index(i)
            end_i=start_i+len(i)-1
            for j in a3:
                try:     
                    start_j=q.index(j)
                except:
                    break
                end_j=start_j+len(j)-1
                
                # 判断交叉
                if (start_j >= start_i and start_j<=end_i) or (end_j >= start_i and end_j<=end_i) or (start_j <= start_i and end_j>=end_i) :
                    if j[0]=='"' and j[-1]=='"':
                        j = j[1:-1]
                    a4.append(j)
                    
        a4_=list(set(a4))
        a4_.sort(key=a4.index)
        ans4.append(a4_)
merged_data = []
# for q,a1,a4 in zip(questions,ans1,ans4):
#     merged_data.append({'q':q,'mentions':a4})
#     print(a1,a4)
# with open(fn_out,'w')as f:
#     json.dump(merged_data,f,ensure_ascii=False)
print(len(ans4))

['成败一知己，生死两妇人'] ['"成败一知己，生死两妇人"', '所说', '的人物', '有什么', '重大成就'] "成败一知己，生死两妇人"所说的人物有什么重大成就？
['茂陵'] ['葬于', '茂陵', '皇帝', '在位于', '时间'] 葬于茂陵的皇帝在位于哪段时间？
['光武中兴', '皇帝'] ['"光武中兴"', '说的是', '皇帝'] "光武中兴"说的是哪位皇帝？
['李清照'] ['李清照', '丈夫的', '代表作品', '是什么'] 李清照丈夫的代表作品是什么？
['兰陵王'] ['"兰陵王"', '是对', '谁的', '尊称'] "兰陵王"是对谁的尊称？
['《西厢记》'] ['《西厢记》', '又名', '什么'] 《西厢记》又名什么？
['白炽灯'] ['白炽灯', '哪国人', '发明'] 白炽灯是哪国人发明的？
['薛宝钗'] ['薛宝钗', '哥哥', '号叫', '什么'] 薛宝钗的哥哥外号叫什么？
['秦桧'] ['史上的', '秦桧', '哪里人'] 历史上的秦桧是哪里人？
['清华大学'] ['清华大学', '的校训', '是什么'] 清华大学的校训是什么？
['严嵩'] ['严嵩', '儿子', '人称'] 严嵩的儿子人称？
['孙悟空'] ['《西游记》', '孙悟空', '兵器', '什么'] 《西游记》中孙悟空的兵器叫什么？
['贾兰'] ['红楼梦》中', '贾兰', '母亲的', '住所', '哪里'] 《红楼梦》中贾兰母亲的住所是哪里？
['唐太宗', '李世民'] ['唐太宗李世民', '年号', '是什么'] 唐太宗李世民的年号是什么？
['东坡肉'] ['东坡肉', '发明人', '死后谥号'] 东坡肉的发明人死后谥号为？
['贾琏'] ['红楼梦》中', '贾琏', '妻子的', '丫鬟', '是谁？'] 《红楼梦》中贾琏妻子的丫鬟是谁？
['乐山大佛'] ['乐山大佛', '始建于', '一年'] 乐山大佛始建于哪一年？
['木牛流马'] ['木牛流马', '发明者', '有哪些', '文学代表作'] 木牛流马的发明者有哪些文学代表作？
['关宁铁骑'] ['创立了', '关宁铁骑'] 谁创立了关宁铁骑？
['明成祖', '朱棣'] ['明成祖朱棣', '年号', '是什么'] 明成祖朱

In [ ]:
for q,a1,a4 in zip(questions,gold_mentions,ans4):
    print(a1,a4)

In [4]:
def evaluate(gold_list,pred_list):
    i=0
    TP = 0 # 预测完全正确
    total_pred = 0
    total_gold = 0
    for gold,pred in zip(gold_list,pred_list):
        if not gold or [] in gold:
            #print(gold,pred)
            continue
        #print(gold, pred)
        i += 1
        TP += len(set(gold) & set(pred))
        total_pred += len(pred)
        total_gold +=len(gold)
    P = 1.0 * TP / total_pred
    R = 1.0 * TP / total_gold
    #print(TP)
    F = 2*P*R/(P+R)
    print(i)
    return P, R, F

print(evaluate(gold_mentions, ans1))
print(evaluate(gold_mentions, ans4))

696
(0.7952218430034129, 0.83115338882283, 0.8127906976744186)
696
(0.7582846003898636, 0.925089179548157, 0.8334226031065882)


### 百科搜索辅助实体链接(可能会报网络连接什么的错误，需要多试几回)

In [8]:
with open(fn_out,'r')as f:
    data = json.load(f)
for one_data in data:
    print(one_data)

{'q': '"成败一知己，生死两妇人"所说的人物有什么重大成就？', 'mentions': ['成败一知己，生死两妇人']}
{'q': '葬于茂陵的皇帝在位于哪段时间？', 'mentions': ['茂陵']}
{'q': '"光武中兴"说的是哪位皇帝？', 'mentions': ['光武中兴', '皇帝']}
{'q': '李清照丈夫的代表作品是什么？', 'mentions': ['李清照']}
{'q': '"兰陵王"是对谁的尊称？', 'mentions': ['兰陵王']}
{'q': '《西厢记》又名什么？', 'mentions': ['《西厢记》']}
{'q': '白炽灯是哪国人发明的？', 'mentions': ['白炽灯']}
{'q': '薛宝钗的哥哥外号叫什么？', 'mentions': ['薛宝钗']}
{'q': '历史上的秦桧是哪里人？', 'mentions': ['秦桧']}
{'q': '清华大学的校训是什么？', 'mentions': ['清华大学']}
{'q': '严嵩的儿子人称？', 'mentions': ['严嵩']}
{'q': '《西游记》中孙悟空的兵器叫什么？', 'mentions': ['孙悟空']}
{'q': '《红楼梦》中贾兰母亲的住所是哪里？', 'mentions': ['贾兰']}
{'q': '唐太宗李世民的年号是什么？', 'mentions': ['唐太宗', '李世民', '唐太宗李世民']}
{'q': '东坡肉的发明人死后谥号为？', 'mentions': ['东坡肉']}
{'q': '《红楼梦》中贾琏妻子的丫鬟是谁？', 'mentions': ['贾琏']}
{'q': '乐山大佛始建于哪一年？', 'mentions': ['乐山大佛']}
{'q': '木牛流马的发明者有哪些文学代表作？', 'mentions': ['木牛流马']}
{'q': '谁创立了关宁铁骑？', 'mentions': ['关宁铁骑']}
{'q': '明成祖朱棣的年号是什么？', 'mentions': ['明成祖', '朱棣', '明成祖朱棣']}
{'q': '武则天的陵寝是？', 'mentions': ['武则天', '武则天的']}
{'q': '《红楼梦》中贾环的父

In [15]:
# 利用百科数据
from tqdm import tqdm
ans6=[]
fn_out_2="data/test_er_out_baike.json"
with open(fn_out,'r')as f:
    data = json.load(f)
for one_data in data:
    q = one_data['q']
    a5 = one_data['mentions']
    a6=a5
    a6_=[]
    for i in a5:   
        if len(i)==1:
            continue
        try:
            out_baike=baike(i).entry
        except:
            out_baike=baike(i).entry
        time.sleep(1)
        if out_baike:
            a6_.append(out_baike)
    if a6_:
        a6.extend(a6_)
        a6=list(set(a6))
    ans6.append({'q':q,'mentions':a6})
    print(q,a6)
with open(fn_out_2,'w')as f:
    json.dump(ans6,f,ensure_ascii=False)    

"成败一知己，生死两妇人"所说的人物有什么重大成就？ ['成败一知己，生死两妇人']
葬于茂陵的皇帝在位于哪段时间？ ['茂陵']
"光武中兴"说的是哪位皇帝？ ['光武中兴', '皇帝']
李清照丈夫的代表作品是什么？ ['李清照']
"兰陵王"是对谁的尊称？ ['高长恭', '兰陵王']
《西厢记》又名什么？ ['崔莺莺待月西厢记', '《西厢记》']
白炽灯是哪国人发明的？ ['白炽灯']
薛宝钗的哥哥外号叫什么？ ['薛宝钗']
历史上的秦桧是哪里人？ ['秦桧']
清华大学的校训是什么？ ['清华大学']
严嵩的儿子人称？ ['严嵩']
《西游记》中孙悟空的兵器叫什么？ ['孙悟空']
《红楼梦》中贾兰母亲的住所是哪里？ ['贾兰']
唐太宗李世民的年号是什么？ ['唐太宗', '唐太宗李世民', '李世民']
东坡肉的发明人死后谥号为？ ['东坡肉']
《红楼梦》中贾琏妻子的丫鬟是谁？ ['贾琏']
乐山大佛始建于哪一年？ ['乐山大佛']
木牛流马的发明者有哪些文学代表作？ ['木牛流马']
谁创立了关宁铁骑？ ['关宁铁骑']
明成祖朱棣的年号是什么？ ['明成祖', '明成祖朱棣', '朱棣']
武则天的陵寝是？ ['武则天', '武则天的']
《红楼梦》中贾环的父亲的人物原型是谁？ ['贾环']
"负荆请罪"说的是谁的故事？ ['负荆请罪']
战国四大名将之首的外号是？ ['战国四大名将之首']
《六韬》的作者有什么重大成就？ ['《六韬》', '六韬']
莫妮卡·贝鲁奇的代表作？ ['莫妮卡·贝鲁奇']
《湖上草》是谁的诗？ ['湖上草']
龙卷风的英文名是什么？ ['龙卷风']
新加坡的水域率是多少？ ['新加坡']
商朝在哪场战役中走向覆灭？ ['商朝']
叔本华信仰什么宗教？ ['叔本华', '亚瑟·叔本华']
大兴安岭的终点是哪里？ ['大兴安岭']
演员梅艳芳有多高？ ['梅艳芳']
被誉为万岛之国的是哪个国家？ ['挪威', '万岛之国']
北京奥运会的口号是什么？ ['北京奥运会', '2008年北京奥运会']
新加坡的政治元老是谁？ ['新加坡']
叶文洁毕业于哪个大学？ ['叶文洁']
"两弹一星"指的是什么？ ['两弹一星']
"超级碗"是什么体育项目联赛的年度冠军赛？ ['超级碗']
清明节起源于哪里？ ['清明节']
诗

《众里寻你》是韩红哪张专辑里的歌? ['韩红', '众里寻你', '《众里寻你》']
久川绫的代表作品有哪些? ['久川绫']
日本的校园和魔法主题漫画有哪些? ['日本的', '校园', '魔法', '日本']
校园卡的英文名是什么? ['校园卡']
购买力平价理论的核心观点是什么? ['购买力平价理论']
蒲树林出品的，由杨幂和季冠霖主演的电视剧是什么？ ['季冠霖', '杨幂', '蒲树林']
主演中有北京外国语大学、北京电影学院和上海戏剧学院毕业生的电视剧有哪些 ['和上', '上海戏剧学院', '北京外国语大学', '戏剧学院', '北京电影学院']
袁弘的身高是？ ['袁弘']
腾讯公司的创始人有哪些？ ['腾讯公司', '腾讯']
位于兴平的有限责任公司有哪些? ['有限责任公司', '兴平']
北京的高等学府有哪些？ ['高等学府', '北京', '北京的']
北京高校的现任校长有哪些? ['高校', '北京', '高等学校']
C9联盟由哪些机构构成？ ['九校联盟', 'C9联盟']
动物界鸟纲画眉科有哪些生物？ ['鸟纲', '画眉科']
台湾山蚂蝗的别称是什么？ ['台湾山蚂蝗']
周润发,黄光亮,梁家辉一同主演的粤语电影有？ ['周润发', '梁家辉', '粤语', '黄光亮']
贾月梅毕业院校地址是？ ['贾月梅']
设有建筑学院的211工程大学的学校代码是？ ['大学的', '211工程', '建筑学院']
出生于邮编为341500地区的汉族官员是？ ['341500']
发作时间随机的蠕虫病毒是？ ['随机', '蠕虫病毒']
河北人创办的企业是？ ['人创办', '河北']
祖籍泉州的诗人创作出的现代诗有？ ['泉州', '诗人', '现代诗']
创作过朦胧诗的人有？ ['朦胧诗']
创作过朦胧诗的老师是？ ['朦胧诗']
62年8月出生的全国劳模有？ ['62', '62年', '8月', '全国劳动模范', '全国劳模', '劳动模范', '劳模']
王后雄和陈国庆合著的书是？ ['陈国庆', '王后雄']
用椰汁和南瓜烹饪的汤有？ ['椰汁', '南瓜']
用西瓜可以制作什么清热去火的饮品是？ ['清热去火', '西瓜']
胡歌的粉丝自称什么？ ['胡歌']
平定大小金川,镇压林爽文起义的清朝将领,死后的谥号是? ['清平大小

招商银行的外部监事 ['招商银行']
长春高新的外部董事 ['长春高新', '长春高新技术产业（集团）股份有限公司']
圣邦股份的市场总监 ['圣邦股份']
中源协和的常务副总经理 ['中源协和']
浔兴股份的总会计书 ['浔兴股份']
浔兴股份的张健群担任了哪些职务 ['浔兴股份', '张健群']
通威股份的总审计师 ['通威股份']
中国太保的总精算师 ['中国太保']
中文在线的执行总经理 ['中文在线']
青岛啤酒的执行董事 ['青岛啤酒']
长城汽车的技术研究院试验中心总工程师 ['长城汽车']
纳思达的技术负责人 ['纳思达']
永兴特钢的经理助理 ['永兴特钢']
铁汉生态的联席总裁 ['铁汉生态']
岭南股份的股东监事 ['岭南股份', '岭南生态文旅股份有限公司']
新华都的董事会办公室主任 ['新华都']
步长制药的采购副总裁 ['山东步长制药有限公司', '步长制药']
步长制药负责采购的总裁是? ['山东步长制药有限公司', '步长制药']
在中金黄金担任财务负责人,总会计师的是谁 ['中金黄金']
中国大陆银行类别 ['中国', '大陆银行']
中央银行的简称 ['中央银行']
中央银行的全称 ['中央银行']
商业银行类别 ['商业银行']
中国五大行 ['五大', '中国']
马云的妻子 ['马云']
阿里巴巴创始人 ['阿里巴巴集团', '阿里巴巴']
鄢建兵的前妻是谁 []
可转换债券的三个特点 ['可转换债券']
北京有几家湘财证券的营业部 ['湘财证券']
建国门湘财证券营业部地址 ['湘财证券', '建国门']
我家在朝阳区，附近有哪几家湘财证券营业部 ['湘财证券', '朝阳区', '朝阳']
朝阳门外大街上有湘财证券的营业部吗 ['湘财证券', '朝阳门外大街']
海淀区的湘财证券营业部叫什么? ['湘财证券']
恒生指数的缩写 ['恒生指数']
MACD指标的中文全称 ['MACD指标']
常用的KDJ参数 ['KDJ指标', 'KDJ']
信用卡类别 ['信用卡']
基金种类 ['基金种类', '基金']
投资者申请赎回基金，几天后可查询 ['基金', '赎回']
基金开户途径 ['基金', '基金开户']
投资银行经营业务 ['投资银行']
投资银行的四种类型 ['投资银行']
证券承销的方式 ['证券承销']
股份

### 实体链接

In [21]:
# 利用百科查找的结果
top=10 # 先取前top10的实体
fn_out="data/test_el_baike_top%s.json"%top

with open(fn_out_2,'r')as f:
    data = json.load(f)
test_EL_data = []
for one_data in data:
    que, cand = one_data['q'], one_data['mentions']
    entities=search_ent(cand,cur,top=100)
    es=[]
    for key in entities.keys():
        es.append(entities[key][:top])
        entities[key]=entities[key][:top]   
    test_EL_data.append({'question':que,'mentions':cand,'ents':entities})
with open(fn_out,'w')as f:
    for item in test_EL_data:
        mystr = json.dumps(item,ensure_ascii=False)
        f.write(mystr+'\n')

### 备用cell，用来测试

In [7]:
# import pickle
# f = open("test_实体识别结果_加长串_baike.pkl","rb")
# out=pickle.load(f)
# num=766
# i=-1
# for q,o in zip(questions,out):
#     i+=1
# #     if i>num/2:
#     print(q,o)
# print(len(out))

In [8]:
#pickle.dump(test_EL_data,open("test_实体链接_加长串_baike_top%s.pkl"%top,'wb'))